In [1]:
import os
import pandas as pd
from openai import OpenAI
from src.func import get_user_watch_history

client = OpenAI()

merged_df = pd.read_csv("data/merged_df.csv")
unique_movie_titles = merged_df['title'].unique().tolist()
system_prompt = "You are a movie recommender system that will compare user previous watch history and ratings."

# def get_movies_and_predict_ratings():
user_response = input("Are you planning to watch the movie alone or with a partner? (alone[a]/partner[p]): ").strip().lower()
if user_response in ['alone', 'a']:
  user_id = int(input("Enter Your User ID:"))
  if user_id in merged_df['userId'].values:
    titles_list, ratings_list = get_user_watch_history(user_id, merged_df)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": f"{system_prompt} Based on the genre and ratings of those movies that this particular user gave, please recommend 10 movie that this user would enjoy watching and predict the rating of these 10 movies given by this user"},
      {"role": "user", "content": f"User {user_id}'s previously watch movies include {titles_list} and the corresponded ratings are {ratings_list}. Please suggest 10 movie based on the watch history and predict ratings of those 10 movies based on user preferences"}
    ]
  )
  else:
    print("This is not a valid user ID, please try again")
elif user_response in ['partner', 'p']:
  user_A = int(input("Enter Your User ID:"))
  user_B = int(input("Enter Your Partner's User ID:"))
  if user_A and user_B in merged_df['userId'].values:
    titles_list_A, ratings_list_A = get_user_watch_history(user_A, merged_df)
    titles_list_B, ratings_list_B = get_user_watch_history(user_B, merged_df)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": f"{system_prompt} Based on the genre and ratings of movies from 2 users, please recommend 10 movie that these 2 users would enjoy watching together and predict the rating of these 10 movies"},
      {"role": "user", "content": f"User {user_A}'s previously watch movies include {titles_list_A} and the corresponded ratings are {ratings_list_A}. User {user_B}'s previously watch movies include {titles_list_B} and the corresponded ratings are {ratings_list_B} Please suggest 10 movie that both users would enjoy watching based on the watch histories and predict specific ratings of those 10 movies based on user preferences"}
    ]
  )
else:
    print("Invalid response. Please answer with 'alone','a' or 'partner','p'.")
    

print(completion.choices[0].message.content) 

Based on User 312's previously watched movies and ratings, I would recommend the following 10 movies that they might enjoy:

1. The Shining (1980)
2. Pulp Fiction (1994)
3. The Godfather (1972)
4. Fight Club (1999)
5. The Matrix (1999)
6. The Dark Knight (2008)
7. Inception (2010)
8. The Silence of the Lambs (1991)
9. Forrest Gump (1994)
10. The Shawshank Redemption (1994)

As for the predicted ratings for these movies, it is difficult to provide an exact prediction without knowing User 312's specific preferences. However, based on the previous ratings they have given, these movies are highly regarded and have generally received positive reviews. Hence, it is likely that User 312 will rate these movies somewhere between 3.5 and 5.0.


# Watch party with 2 userID
user_A = 23

user_B = 445

In [2]:
print(completion.choices[0].message.content)

Based on User 312's previously watched movies and ratings, I would recommend the following 10 movies that they might enjoy:

1. The Shining (1980)
2. Pulp Fiction (1994)
3. The Godfather (1972)
4. Fight Club (1999)
5. The Matrix (1999)
6. The Dark Knight (2008)
7. Inception (2010)
8. The Silence of the Lambs (1991)
9. Forrest Gump (1994)
10. The Shawshank Redemption (1994)

As for the predicted ratings for these movies, it is difficult to provide an exact prediction without knowing User 312's specific preferences. However, based on the previous ratings they have given, these movies are highly regarded and have generally received positive reviews. Hence, it is likely that User 312 will rate these movies somewhere between 3.5 and 5.0.


# Recommend based on genre

In [3]:
from src.func import find_top_3_genre

# def get_movie_genre():
user_id_genre = int(input("We will suggest movies based on your favorite genre, please enter your user ID:"))
if user_id_genre in merged_df['userId'].values:
    top_3_genre = find_top_3_genre(user_id_genre,merged_df)
    genre_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a movie recommender system that will recommend 10 movie in this user's favorite genres and predict the rating of these 10 movies given by this user"},
      {"role": "user", "content": f"User {user_id_genre}'s favorite genre include {top_3_genre}. Please suggest 10 movie in these genre for the user"}
    ]
)
else:
    print("This is not a valid user ID, please try again")

print(genre_response.choices[0].message.content)

Based on User 231's favorite genres (Crime, Action, Horror), here are 10 movie recommendations:

1. "The Dark Knight" - Genre: Crime, Action
2. "Pulp Fiction" - Genre: Crime, Action
3. "Se7en" - Genre: Crime, Horror
4. "The Departed" - Genre: Crime, Action
5. "Get Out" - Genre: Horror, Mystery
6. "The Silence of the Lambs" - Genre: Crime, Horror
7. "Kill Bill: Volume 1" - Genre: Crime, Action
8. "A Quiet Place" - Genre: Horror, Thriller
9. "Training Day" - Genre: Crime, Action
10. "The Conjuring" - Genre: Horror, Mystery

Please note that these recommendations are based on the user's favorite genres and may vary depending on individual preferences.


# Recommend based on similar user

In [4]:
from src.func import get_similar_users

# def rec_from_similar_user():
user_id_sim = int(input("We will suggest movies you may also like based on similar user, please enter your user ID:"))
if user_id_sim in merged_df['userId'].values:
    most_similar = get_similar_users(user_id_sim, merged_df)
    titles_list, ratings_list = get_user_watch_history(most_similar, merged_df)
    response_sim = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "You are a movie recommender system that will suggest movies that the user may also like based on similar user's watch histories. From the most similar user's watch histories, please recommend 10 movies that this user would enjoy watching and predict the rating of these 10 movies given by this user"},
    {"role": "user", "content": f"User {user_id_sim}'s most similar user is user {most_similar}, which has previously watched {titles_list} and the corresponded ratings are {ratings_list}. Please suggest 10 movies that user {user_id_sim} may also like and provide rating prediction of those 10 movies based on user preferences"}
    ]
)
else:
    print("This is not a valid user ID, please try again")   

print(response_sim.choices[0].message.content)

To recommend 10 movies for user 323, we can consider the movies watched by user 38 and their corresponding ratings. Based on the similarity between the two users, there is a higher chance that user 323 will also enjoy these movies. Here are the 10 movie recommendations for user 323:

1. Legends of the Fall (1994) - Predicted Rating: 4.0
2. Usual Suspects, The (1995) - Predicted Rating: 4.5
3. Quiz Show (1994) - Predicted Rating: 4.0
4. Schindler's List (1993) - Predicted Rating: 5.0
5. Jurassic Park (1993) - Predicted Rating: 4.0
6. Apollo 13 (1995) - Predicted Rating: 4.0
7. Clerks (1994) - Predicted Rating: 3.5
8. True Romance (1993) - Predicted Rating: 3.5
9. Forrest Gump (1994) - Predicted Rating: 4.5
10. Pulp Fiction (1994) - Predicted Rating: 4.5

Please note that the predicted ratings are based on the ratings provided by user 38 for the respective movies.


# Rating prediction for evaluation metric

In [11]:
from src.func import get_user_watch_history_20
# def rating_prediction()
user_id_rating = int(input("We want to predict movie ratings from your watch list, please enter your user ID:"))
if user_id_rating in merged_df['userId'].values:
    titles_list, ratings_list = get_user_watch_history(user_id_rating, merged_df)
    test_title_list, true_ratings = get_user_watch_history_20(user_id_rating, merged_df, fraction=0.8)
    response_rating = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "You are a movie rating prediction system that will predict rating with a list of titles given"},
    {"role": "user", "content": f"User {user_id_rating}'s previously watch movies include {titles_list} and the corresponded ratings are {ratings_list}. Based on these ratings, please provide a list of prediction rating corresponded to {test_title_list} with the format of movie title - Predicted Rating: ratings"}
    ]
)
else:
    print("This is not a valid user ID, please try again")   

print(response_rating.choices[0].message.content)

Here are the predicted ratings for the given movies:

- Alien (1979) - Predicted Rating: 4.0
- Terminator, The (1984) - Predicted Rating: 4.5
- Matrix, The (1999) - Predicted Rating: 4.5
- Interstellar (2014) - Predicted Rating: 4.5
- Godfather, The (1972) - Predicted Rating: 5.0


# Evaluation metric (RMSE of the prediction)

In [12]:
from src.func import extract_ratings
message = f"{response_rating.choices[0].message.content}"
movies_and_ratings = extract_ratings(message)
predicted_ratings = []
for movie, rating in movies_and_ratings:
    predicted_ratings.append(rating)

In [13]:
len(true_ratings)

5

In [14]:
import math
from src.func import calculate_rmse
rmse_score = calculate_rmse(predicted_ratings, true_ratings)
print(f"RMSE: {rmse_score}")

RMSE: 0.8660254037844386


# To Do List
- Organized prompts into src.function or class
- put into .py file
- Think about evaluation metric to compare baseline and message from prompt
- requirement.txt file
- update readme